<a href="https://colab.research.google.com/github/joony0512/Deep_Learning_Class/blob/main/Part5/P5_Ch06_CH06_06_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN SeqSeq

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/#fastcampus
!pwd

!ls

/content/drive/MyDrive/#fastcampus
/content/drive/MyDrive/#fastcampus
 config_utils.py      data_utils.py   requirements.txt
 config_utils_tf.py   __init__.py     runs
 config.yaml	      logs	     'runs(torch)'
 custom_math.py       models	      wandb
 data		      __pycache__    '[강의자료]-노트북-ipynb-파일'


In [3]:
import os
import sys
sys.path.append('/content/drive/MyDrive/#fastcampus')
drive_project_root = '/content/drive/MyDrive/#fastcampus'
!pip install -r '/content/drive/MyDrive/#fastcampus/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.4/722.4 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.6/144.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 32.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.6/222.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.4/435.4 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 k

In [4]:
!pip install tensorflow-addons
!pip install wandb
!pip install omegaconf
!pip install hydra-core==1.1


  Using cached tensorflow_addons-0.21.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (612 kB)
  Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)
  Using cached wandb-0.15.5-py3-none-any.whl (2.1 MB)
  Using cached GitPython-3.1.32-py3-none-any.whl (188 kB)
  Using cached sentry_sdk-1.28.1-py2.py3-none-any.whl (214 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached setproctitle-1.3.2-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=5d206

  Using cached hydra_core-1.1.0-py3-none-any.whl (144 kB)
  Using cached omegaconf-2.1.2-py3-none-any.whl (74 kB)
  Using cached antlr4_python3_runtime-4.8-py3-none-any.whl
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.9.3
    Uninstalling antlr4-python3-runtime-4.9.3:
      Successfully uninstalled antlr4-python3-runtime-4.9.3
  Attempting uninstall: omegaconf
    Found existing installation: omegaconf 2.3.0
    Uninstalling omegaconf-2.3.0:
      Successfully uninstalled omegaconf-2.3.0


In [5]:
from typing import Optional
from typing import List
from typing import Dict
from typing import Tuple

import io
import re
import unicodedata
import time
from datetime import datetime

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.pyplot as ticker
from omegaconf import OmegaConf
from omegaconf import DictConfig
import hydra
from hydra.core.config_store import ConfigStore

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

import wandb


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [6]:
from data_utils import dataset_split
from config_utils_tf import flatten_dict
from config_utils_tf import register_config
from config_utils_tf import get_optimizer_element
from config_utils_tf import get_callbacks


In [7]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
!nvidia-smi

Sat Jul 15 07:40:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 데이터 및 데이터로더 정의 및 전처리

In [9]:
# Download the file
data_root = os.path.join(drive_project_root, "data", "anki_spa_eng")
if not os.path.exists(data_root):
    os.mkdir(data_root)

data_path = os.path.join(data_root, "spa-eng.zip")

path_to_zip = tf.keras.utils.get_file(
    data_path,
    origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True,
    cache_dir = data_root)

path_to_file = os.path.join(
    os.path.dirname(path_to_zip),
    "datasets",
    "spa-eng",
    "spa.txt",
)

print(path_to_file)

/content/drive/MyDrive/#fastcampus/data/anki_spa_eng/datasets/spa-eng/spa.txt


In [21]:
# 전처리

def unicode_to_ascii(s):
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) !="Mn")

def preprocessing_sentence(w):
    # ascii 로 변환 및 소문자로 변환
    w = unicode_to_ascii(w.lower().strip())
    # 단어와 단어 뒤에 오는 구두점(.) 사이에 공백을 생성
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # (a-z, A-Z, [?.!,¿] 을 제외한 모든 것을 공백으로 대체)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.strip()

    # 모델의 앞뒤에 start, end 토큰 추가
    w = "<start> "+ w +" <end>"
    return w

def create_dataset(path : str, num_examples : Optional[int]=None):
    lines = io.open(path, encoding = "UTF-8").read().strip().split("\n")

    word_pairs =[[preprocessing_sentence(w) for w in l.split("\t")] for l in lines[:num_examples] ]
    # None인경우 다가져오고 숫자가 있는경우 숫자만큼만 가져온다.
    return zip(*word_pairs)


en, sp = create_dataset(path_to_file)
print(en[-1])
print(sp[-1])

<start> if you want to sound like a native speaker , you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo . <end>
<start> si quieres sonar como un hablante nativo , debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un musico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado . <end>


In [22]:
# Tokenizer 정의, 최종적으로 쓸 데이터 정의.

def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters ="")
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = pad_sequences(tensor, padding = 'post')

    return tensor, lang_tokenizer

def load_dataset(path, num_examples = None):

    tar_lang, src_lang = create_dataset(path, num_examples) # en, sp

    src_tensor, src_tokenizer = tokenize(src_lang)
    tar_tensor, tar_tokenizer = tokenize(tar_lang)

    return src_tensor, tar_tensor, src_tokenizer,tar_tokenizer

# 언어 데이터셋을 불러오기
num_examples = 30000
src_tensor, tar_tensor, src_tokenizer,tar_tokenizer = load_dataset(
    path_to_file, num_examples
)

mar_tar_len, mar_src_len = tar_tensor.shape[1], src_tensor.shape[1]

src_vocab_size = len(src_tokenizer.word_index) +1
tar_vocab_size = len(tar_tokenizer.word_index) +1

print(src_vocab_size)
print(tar_vocab_size)


9414
4935


In [23]:
print(tar_tensor[-2])
print(tar_tokenizer.word_index)

for i in tar_tensor[-2]:
    if i ==0:
        break
    print(i, tar_tokenizer.index_word[i])

[  1  16  38 369 107  39   3   2   0   0   0]
{'<start>': 1, '<end>': 2, '.': 3, 'i': 4, 'tom': 5, 'you': 6, '?': 7, 'is': 8, 'a': 9, 'it': 10, 's': 11, 't': 12, 'the': 13, 'he': 14, 'to': 15, 'we': 16, 'me': 17, 'm': 18, 'this': 19, 'that': 20, 'my': 21, 'do': 22, 're': 23, 'are': 24, 'can': 25, 'was': 26, 'she': 27, 'they': 28, 'have': 29, 'don': 30, 'your': 31, 'what': 32, 'in': 33, 'not': 34, 'like': 35, 'go': 36, '!': 37, 'll': 38, 'here': 39, 'be': 40, 'him': 41, 'did': 42, 'know': 43, 'on': 44, 'mary': 45, 'let': 46, 'want': 47, 'very': 48, ',': 49, 'up': 50, 'has': 51, 'how': 52, 'come': 53, 'for': 54, 'at': 55, 'please': 56, 'there': 57, 'now': 58, 'of': 59, 'who': 60, 'his': 61, 'am': 62, 'need': 63, 'get': 64, 've': 65, 'no': 66, 'all': 67, 'her': 68, 'good': 69, 'just': 70, 'where': 71, 'see': 72, 'help': 73, 'out': 74, 'will': 75, 'got': 76, 'love': 77, 'too': 78, 'an': 79, 'with': 80, 'us': 81, 'why': 82, 'car': 83, 'one': 84, 'so': 85, 'stop': 86, 'didn': 87, 'look': 88,

## 모델정의


In [ ]:
class GRUEncoder(tf.keras.Model):
    def __init__(self, cfg: DictConfig):
        super.__init__()
        self.cfg = cfg
    def call(self, src_tokens, state = None, training =False):
        pass

class GRUDecoder(tf.keras.Model):
    def __init__(self, cfg : DictConfig):
        super.__init__()
        self.cfg = cfg
    def call(self, tar_tokens, state= None, training = False):
        pass